# MC886 Assigment 1

## Objective:
Use linear regression model to predict the prices of diamonds given their atributes. The data set might be found on:
https://www.kaggle.com/shivam2503/diamonds.


In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from utils import *

In [2]:
data_dir = '../data/diamonds.csv'

In [3]:
with open(data_dir, 'rb') as csvfile:
    dataset = pd.read_csv(data_dir)
    
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [4]:
#carat	cut	color	clarity	depth	table	price	x	y	z
# print(dataset['cut'])

## Preprocessing the data

### 1. Split data in training, validation and test
A wise person said: "friends don’t let friends use testing data for training". t.

**Note:**
* **sklearn.model_selection.train_test_split(*arrays, **options)**  
    Split arrays or matrices into random train and test subsets.  
    See documentation:http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

### 2. Put the dataset into Numpy volume
Categorical atributes are changed to integers.  
The cell bellow runs a code to shuffle and split the dataset into 3 sets, training, validation and test.

### 3. Transform categorical data into numbers
Used fit_transform function.
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
http://pbpython.com/categorical-encoding.html

In [5]:
#RUN THIS ONLY ONCE!
# this code saves the numpy vectors to files
# split_data(dataset)

In [11]:
# lb_make = LabelEncoder()
# print(dataset.shape)
# column = np.empty((dataset.shape[0],1))
# column= lb_make.fit_transform(dataset['cut'])
# # print(dataset['cut'])
# print(dataset['carat'])

(53940, 11)
0        0.23
1        0.21
2        0.23
3        0.29
4        0.31
5        0.24
6        0.24
7        0.26
8        0.22
9        0.23
10       0.30
11       0.23
12       0.22
13       0.31
14       0.20
15       0.32
16       0.30
17       0.30
18       0.30
19       0.30
20       0.30
21       0.23
22       0.23
23       0.31
24       0.31
25       0.23
26       0.24
27       0.30
28       0.23
29       0.23
         ... 
53910    0.70
53911    0.57
53912    0.61
53913    0.80
53914    0.84
53915    0.77
53916    0.74
53917    0.90
53918    0.76
53919    0.76
53920    0.70
53921    0.70
53922    0.70
53923    0.73
53924    0.73
53925    0.79
53926    0.71
53927    0.79
53928    0.79
53929    0.71
53930    0.71
53931    0.71
53932    0.70
53933    0.70
53934    0.72
53935    0.72
53936    0.72
53937    0.70
53938    0.86
53939    0.75
Name: carat, Length: 53940, dtype: float64


In [ ]:
#load the numpy files (training set, validation set, test set)
x_train, y_train, x_validation, y_validation, x_test, y_test = load_vectors()
y_train = y_train.reshape((1,y_train.shape[0]))
y_validation = y_validation.reshape((1,y_validation.shape[0]))
y_test = y_test.reshape((1,y_test.shape[0]))
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
def init_parameters(X):
    """
    Receives: the input vector shape (nx, m), where nx is the number of features and
    m the number of examples.
    Returns:
    W: the weight array shape ( 1, nx)
    dW: the derivative array shape ( 1, nx)
    b: a real number (bias)
    db: the derivative of the bias.
    """
    nx = X.shape[0]
    W = np.random.randn(nx,1)*0.01
    dw = np.random.randn(nx,1)*0.01
    b = 0.
    db = 0.
    return W,b, dw, db

In [ ]:
def l2_cost(X, Y, Y_hat, m):
    """
    Cost function defined by a variation of the l2 norm function.
    The '2' in the denominator makes its deriative easier
    """
    cost = (1/(2*m))*np.sum((Y-Y_hat)**2)
    return cost

In [ ]:
#FORWARD PROPAGATION (WELL THIS IS NOT A NN, BUT ...)
def forward_prop(X, w, b):
    """
    X is the input vector with m training examples, shape (1,m)
    w: is the angular coefficient shape (9,1)
    b: is the linear coefficient  shape (9,1)
    """
    Y_hat = np.dot(w.T, X) + b
    return Y_hat

In [ ]:
def backward_propagation(X,Y, Y_hat, m):
    """
    Calculates the derivatives of the parameters
    returns:
    dw
    db
    """
    nx = X.shape[0]
    ny = Y.shape[0]
    dw = (-1./m)*np.dot(X,(Y -Y_hat).T)
    db = (-1./m)*np.sum((Y -Y_hat))
    assert(dw.shape == (nx, ny))
    return dw, db

In [ ]:
def GD_model(X,Y, num_iterations = 100, learning_rate = 0.000000001):
    """
    Arguments:
    X: input array shape (nx, m), nx is the number of input features and m the number of
    training examples.
    Y: label array shape (ny, m) where ny is the number of the output feature.
    Given X and Y estimates the best linear function to fit the data
    """
    m = X.shape[1]
    print('m = ', m) 
    
    w,b,dw,db = init_parameters(x_train)
    for iteration in range(num_iterations):
        
        #forward propagation (we can think the activation function as linear)
        Y_hat = forward_prop(X,w,b)
        
        #plot the cost function
        cost = l2_cost(X,Y, Y_hat,m)
        plt.plot(iteration,cost,'ro')
    
        #Backward propagation calculate the derivatives 
        dw, db = backward_propagation(X,Y,Y_hat, m)
    
        #Update parameters simultaneusly
        temp_w = w - learning_rate*dw
        temp_b = b - learning_rate*db
        w = temp_w
        b = temp_b
    plt.show()
    print("dw.shape = ", dw.shape)
    print("db.shape = ", db.shape)
    print("w.shape = ", w.shape)
    return w, b, cost

In [ ]:
w_gd, b_gd, cost_gd= GD_model(x_train,y_train)
print("cost in the training set: ", cost_gd)
print("weights: ", w_gd)
print("bias: ", b_gd)

In [ ]:
def SGD_model(X,Y, num_iterations = 20, learning_rate = 0.00000001):
    """
    X: an input array shape (nx, m), nx is the input feature size and m is the number of training examples
    Given X and Y estimates the best linear function to fit the data
    """
    # initialize parameters
    w,b,dw,db = init_parameters(x_train)
    print("first ", dw)
    m = X.shape[1]    # number of training examples
    nx = X.shape[0]   # number of features in the input
    ny = Y.shape[0]   # number of features in the label
    
    # for each epoch
    for iteration in range(num_iterations):
        print("\riteração: ", iteration +1, end = "")
        #for each element in the example's vector (shape (9,1))
        Y_hat = np.empty((ny, m))
        for i in range(m):
            #extract a columns of the input, remember the ith column has the ith example
            x_i = X[:, i].reshape(nx,1) #reshape to avoid rank 1 array
            y_i = Y[:, i].reshape(ny,1)
#             print("x_i.shape = ", x_i.shape)
#             print("y_i.shape = ", y_i.shape)
#             print("w.shape = ", w.shape)
            
            #calulate the prediction y_hat
            y_hat_i = np.dot(w.T,x_i) + b
            Y_hat[:,i] = y_hat_i
#             print("y_hat_i.shape = ", y_hat_i.shape)
            
#             #calculate the derivatives
#              dw = (-1./m)*np.dot(X,(Y -Y_hat).T)
#     db = (-1./m)*np.sum((Y -Y_hat))
            
#             dw = 
            dw, db = backward_propagation(x_i,y_i,y_hat_i, 1)
#             print("dw.shape = ", dw.shape)
#             print("db.shape = ", db.shape)
            #Update parameters simultaneusly
            temp_w = w - learning_rate*dw
            temp_b = b - learning_rate*db
            w = temp_w
            b = temp_b
            #plot the cost function
#             cost = (1./(2.))*np.sum((label -Y_hat)**2)
        cost = l2_cost(X,Y, Y_hat,m)
        print("cost = ", cost)
        plt.plot(iteration,cost,'ro')
    plt.show()
    return w, b, cost

In [ ]:
w_sgd, b_sgd, cost_sgd = SGD_model(x_train,y_train)
print("cost in the training set: ", cost_sgd)
print("weights: ", w_sgd)
print("bias: ", b_sgd)

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
clf = linear_model.SGDRegressor(loss='squared_loss', penalty='l2', learning_rate = 'constant', eta0 = 0.00000001)
clf.fit(x_train.T, y_train.T)
# Make predictions using the testing set
y_hat = clf.predict(x_train.T)
# The coefficients
print('Coefficients: \n', clf.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_train.T, y_hat.T))
train_size = 30000
print("Error: %.2f"
      % l2_cost(x_train, y_train, y_hat, train_size))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_train.T, y_hat.T))

# cost [element] = l2_cost(x_train, y_train, y_)
# plt.show()
# print(cost)

## Normal equation

In [ ]:
# X_norm = np.concatenate((x_train, x_validation), axis=1).T

x0_norm = np.ones((1, train_size))
X_norm = np.concatenate((x0_norm, x_train), axis=0)
print(X_norm.shape)
# y_norm = np.concatenate((y_train, y_validation), axis=1).T
# n_features = X_norm.shape[1]
# print('X:', X_norm.shape)
# # compute Normal Eq.
XTX_inv = np.linalg.inv(np.matmul(X_norm.T, X_norm))
theta_norm = np.matmul(np.matmul(XTX_inv, X_norm.T), y_train)
# print('theta: ', theta_norm.shape)

In [ ]:
# Test
X_test_norm = np.concatenate((np.ones((x_test.shape[1], 1)), x_test.T), axis=1)
y_test_norm = y_test

print('test')
print('X test:', X_test_norm.shape)
print('y test:', y_test_norm.shape)
print('*' * 40)

max_diff = 0
for i in range(len(X_test_norm)):
    estimate = np.matmul(theta_norm.T, np.reshape(X_test_norm[i], (n_features, 1)))
    print('i: %d; estimado: %lf; real: %lf; diferenca: %lf' 
          %(i, estimate, y_test_norm[:,i], estimate-y_test_norm[:,i]))

In [ ]:
def hypotesis(theta, X):
    return np.matmul(theta.T, X)

In [ ]:
batch_size = 64
n_epoch = 100
batches_per_epoch = int(np.ceil(train_size / float(batch_size)))
n_steps = int(round(batches_per_epoch * n_epoch))
learning_rate = 0.000000001

print('training size: ', train_size)
print('number of epoch: ', n_epoch)
print('batches per epoch: ', batches_per_epoch)
print('number of steps: ', n_steps)

In [ ]:
#
theta_mbgd = np.random.rand(nx + 1, ny)
x0_mbgd = np.ones((train_size, 1))
X_mbgd = np.concatenate((x0_mbgd, x_train.T), axis=1)
print(theta_mbgd.shape,  X_mbgd.shape)

for j in range(10):
#     np.random.shuffle(X_mbgd)
    for i in range(n_epoch):
        h = hypotesis(theta_mbgd, X_mbgd[i:i+batch_size, :].T)
    #    print(h.shape)
        cost = np.sum((h - np.sum(y_train[:, i:i+batch_size]))) / batch_size

        # update
        theta_mbgd = theta_mbgd - (learning_rate  * cost)
#         if j%10000==0 and i == 0:
        print('j: %d;\tcost: %.3lf' %(j, cost))

In [ ]:
# Test
X_test_mbgd = np.concatenate((np.ones((x_test.shape[1], 1)), x_test.T), axis=1)
y_test_mbgd = y_test

print('test')
print('X test:', X_test_mbgd.shape)
print('y test:', y_test_mbgd.shape)
print('*' * 40)

max_diff = 0
for i in range(len(X_test_mbgd)):
    estimate = np.matmul(theta_mbgd.T, np.reshape(X_test_mbgd[i], (nx+1, 1)))
    print('i: %d; estimado: %.3lf; real: %.3lf; diferenca: %.3lf' 
          %(i, estimate, y_test_mbgd[:,i], estimate-y_test_mbgd[:,i]))